
# Fitting using reference spectra

With these few cells, you can use 2 reference spectra to fit your spectrum images and thus make more accurate quantifications of your data. The reference spectra have to share the same energy scale as the data.

## Create a custom component.

Hyperspy supports the creation of custom components. This is a powerful tool to design your own data analysis.

For this component, the first two arguments ref_1 and ref_2 are the reference spectra (as numpy arrays). They will not change during the fitting procedure. The last arguments a_ref_1 and a_ref_2 are the learned weights of ref_1 and ref_2, respectively.


In [ ]:
from hyperspy.component import Component
import hyperspy.api as hs

class ReferenceFitting(Component):
    def __init__(self, ref_1 = None, ref_2 = None, a_ref_1=1, a_ref_2=2):
        # Define the parameters
        # Note that the reference spectra are not parameters
        Component.__init__(self, ('a_ref_1', 'a_ref_2'))
        
        self.a_ref_1.value = a_ref_1
        self.a_ref_2.value = a_ref_2

        self.a_ref_1.bmin = 0.
        self.a_ref_1.bmax = None
        
        self.a_ref_2.bmin = 0.
        self.a_ref_2.bmax = None
        
        self.a_ref_1.grad = self.a_ref_1
        self.a_ref_2.grad = self.a_ref_2
        
        assert ref_1 is not None, "You need to define a first reference spectrum."
        assert ref_2 is not None, "You need to define a second reference spectrum."
        
        self.ref_1 = ref_1
        self.ref_2 = ref_2

    # Define the function as a function of the already defined parameters,
    # x being the independent variable value (e.g. energy scale)
    # In this particular case, there is no independant variable
    def function(self, x):
        a1 = self.a_ref_1.value
        a2 = self.a_ref_2.value
        return a1*self.ref_1 + a2*self.ref_2
    
    # Optionally, for faster execution, you can define the gradients of each parameter
    def grad_a_ref_1(self, x):
        """
        Returns d(function)/d(a_ref_1)
        """
        return self.ref_1

    def grad_a_ref_2(self, x):
        """
        Returns d(function)/d(a_ref_2)
        """
        return self.ref_2

## Loading the data and initializing model

The fitting of the edges are disabled so that there is no conflict with the reference spectra.

The ReferenceFitting component is added to the list of components of the model.


In [ ]:
ref1 = hs.load()
ref2 = hs.load()
spim = hs.load()

comp = ReferenceFitting(ref1.data,ref2.data,1,1)
model = spim.create_model()

model.disable_edges()
model.extend([comp])

In [ ]:
# /!\ This can take a while /!\
model.multifit()

In [ ]:
model.plot_results()

In [ ]:
model.save('my_mlls.hspy')